# read sheet rows

In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [2]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']


In [3]:
# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ'
RANGE_NAME = 'opportunities_looker!A1:E'


In [5]:
"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=474716112864-hh2gs0lrd1ie9ja4k18nnnjttceeg8og.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50221%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=ckasjdyUsyV2SmejnlhTXVw98zhN4l&access_type=offline&code_challenge=dkNcVixQw1mc210NkX-082htP7tu3hAjeYCG9dqL1EI&code_challenge_method=S256


In [7]:
# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=RANGE_NAME).execute()
values = result.get('values', [])


In [16]:
sheet_result = sheet.get(spreadsheetId=SPREADSHEET_ID).execute()

In [18]:
sheet_result

{'spreadsheetId': '1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ',
 'properties': {'title': 'BDR Meeting KPIs and Opportunity Targets',
  'locale': 'en_US',
  'autoRecalc': 'ON_CHANGE',
  'timeZone': 'America/Los_Angeles',
  'defaultFormat': {'backgroundColor': {'red': 1, 'green': 1, 'blue': 1},
   'padding': {'top': 2, 'right': 3, 'bottom': 2, 'left': 3},
   'verticalAlignment': 'BOTTOM',
   'wrapStrategy': 'OVERFLOW_CELL',
   'textFormat': {'foregroundColor': {},
    'fontFamily': 'arial,sans,sans-serif',
    'fontSize': 10,
    'bold': False,
    'italic': False,
    'strikethrough': False,
    'underline': False}}},
 'sheets': [{'properties': {'sheetId': 0,
    'title': 'Meetings',
    'index': 0,
    'sheetType': 'GRID',
    'gridProperties': {'rowCount': 996,
     'columnCount': 27,
     'frozenRowCount': 1}},
   'basicFilter': {'range': {'startRowIndex': 0,
     'endRowIndex': 12,
     'startColumnIndex': 0,
     'endColumnIndex': 11},
    'criteria': {'0': {}}}},
  {'properties'

In [9]:
if not values:
    print('No data found.')
else:
    print(values)


[['GEO', 'USER ID', 'BDR', 'Number of Oppty', 'Month'], ['EMEA', '0050f000009neAi', 'Damian Dziengo', '5', '5/1/2019'], ['EMEA', '0050f000009neAi', 'Damian Dziengo', '5', '6/1/2019'], ['EMEA', '0050f000009neAi', 'Damian Dziengo', '5', '7/1/2019'], ['EMEA', '0050f000009neAi', 'Damian Dziengo', '5', '8/1/2019'], ['EMEA', '0050f000009neAi', 'Damian Dziengo', '5', '9/1/2019'], ['EMEA', '0050f000009neAi', 'Damian Dziengo', '5', '10/1/2019'], ['EMEA', '0050f000009neAi', 'Damian Dziengo', '5', '11/1/2019'], ['EMEA', '0050f000009neAi', 'Damian Dziengo', '5', '12/1/2019'], ['AMER', '0050f000009LDlZ', 'Kevin Mununga', '6', '5/1/2019'], ['AMER', '0050f000009LDlZ', 'Kevin Mununga', '6', '6/1/2019'], ['AMER', '0050f000009LDlZ', 'Kevin Mununga', '6', '7/1/2019'], ['AMER', '0050f000009LDlZ', 'Kevin Mununga', '6', '8/1/2019'], ['AMER', '0050f000009LDlZ', 'Kevin Mununga', '6', '9/1/2019'], ['AMER', '0050f000009LDlZ', 'Kevin Mununga', '6', '10/1/2019'], ['AMER', '0050f000009LDlZ', 'Kevin Mununga', '6', 

# change logs for drive

In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
SCOPES = ['https://www.googleapis.com/auth/drive.file']


In [7]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'gsuite_credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)
        

In [22]:
# Call the Drive v3 API
results = service.files().list(
    pageSize=100, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        if item['name'] in ('BDR Meeting KPIs and Opportunity Targets', 'Finance Metrics'):
            print(u'{0} ({1})'.format(item['name'], item['id']))
        

Files:
BDR Meeting KPIs and Opportunity Targets (1mcq90dK81WKiwgk9kczRyCccb6VO2H11kT2d5RoS8iQ)
Finance Metrics (1yRrgnEdujAsSjIDQEYIVmAHluvYVNBtw9nucoLJ7o9w)


In [10]:
response = service.changes().getStartPageToken().execute()

In [11]:
response.get('startPageToken')

'747'

In [15]:
page_token = '747'
while page_token is not None:
    response = service.changes().list(pageToken=page_token,
                                            spaces='drive').execute()
    for change in response.get('changes'):
        # Process change
        print('Change found for file: %s' % change.get('fileId'))
    if 'newStartPageToken' in response:
        # Last page, save this token for the next polling interval
        saved_start_page_token = response.get('newStartPageToken')
    page_token = response.get('nextPageToken')

In [16]:
response

{'kind': 'drive#changeList', 'newStartPageToken': '747', 'changes': []}

# moving to file level (change logs)